In [51]:
import pandas as pd
# se combinan datos de metereologia y particulas contaminantes y se calcula icap en base a pm10

In [52]:
d_pm25 = pd.read_csv('../data/processed/pm_2.5.csv',sep=';', thousands=None, decimal=',')
d_pm10 = pd.read_csv('../data/processed/pm_10.csv',sep=';', thousands=None, decimal=',')

Metereologia = pd.read_csv('../data/processed/Metereologia.csv',sep=';', thousands=None, decimal=',')


In [53]:
d_contaminantes = pd.merge(d_pm25, d_pm10, on='fecha', how='outer')

nuevos_nombres = {
    'registro_x': 'pm2.5',
    'registro_y': 'pm10'
}
d_contaminantes = d_contaminantes.rename(columns=nuevos_nombres)
# Eliminando la fila con índice 0
d_contaminantes.drop(index=0, inplace=True)
d_contaminantes.reset_index(drop=True, inplace=True)

In [54]:
Metereologia.rename(columns={'ff_Valor': 'viento'}, inplace=True)
Metereologia.rename(columns={'dd_Valor': 'dir_viento'}, inplace=True)

DataSet = pd.merge(d_contaminantes, Metereologia, on='fecha', how='outer')

In [55]:
# ETIQUETACION DE CALIDAD DEL AIRE BASADO EN ICAP
# ICAP            MP10 µg/m3N (24 horas)
# 0                0
# 100              150
# 500              330


def ICAP(PM10):

    if PM10 < 0:
        raise ValueError("PM10 debe ser mayor igual a 0")
    

    if 0 <= PM10 <= 150:
        ICAP = PM10 / 1.5
    elif 150 <PM10 :
        ICAP = (PM10 - 114) / 0.36

    
    return ICAP

DataSet['icap'] = DataSet['pm10'].apply(ICAP)

# Definir los bordes de los bins (contenedores)
bins = [-1, 99, 199, 299, 499, float('inf')]

# Definir las etiquetas para los bins. Nota: Las etiquetas se aplican desde el valor más alto al más bajo
labels = [5, 4, 3, 2, 1]

# Crear la nueva columna
DataSet['calidad_aire'] = pd.cut(DataSet['icap'], bins=bins, labels=labels)

# Si quieres que 'calidad_aire' sea de tipo int en lugar de category:
DataSet['calidad_aire'] = DataSet['calidad_aire'].astype(int)
conteo_calidad_aire = DataSet['calidad_aire'].value_counts()
etiquetas = {
    5: "Bueno",
    4: "Regular",
    3: "Alerta ambiental",
    2: "Preemergencia ambiental",
    1: "Emergencia ambiental"
}

for etiqueta_num, etiqueta_texto in etiquetas.items():
    conteo = conteo_calidad_aire[etiqueta_num]
    print(f"El conteo de calidad '{etiqueta_texto}' es: {conteo}")
    
# DataSet.describe()

El conteo de calidad 'Bueno' es: 3655
El conteo de calidad 'Regular' es: 59
El conteo de calidad 'Alerta ambiental' es: 14
El conteo de calidad 'Preemergencia ambiental' es: 11
El conteo de calidad 'Emergencia ambiental' es: 3


In [56]:
nombre = 'Dataset_Particulas_contaminantes'

path = '../data/processed/{}.csv'.format(nombre)


DataSet.to_csv(path, sep=';',index=False, float_format='%.4f', decimal=',')